In [1]:
 from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
spark

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.apache.iceberg#iceberg-aws-bundle added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-277898d4-03cb-4e41-9578-94ac40e302ae;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.10.0 in central
	found org.apache.iceberg#iceberg-aws-bundle;1.10.0 in central
	found org.postgresql#postgresql;42.7.3 in central
	found org.checkerframework#checker-qual;3.42.0 in central
:: resolution report :: resolve 256ms :: artifacts dl 9ms
	:: modules in use:
	org.apache.iceberg#iceberg-aws-bundle;1.10.0 from central in [default]
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.10.0 from central in [default]
	org.checkerframework#checker-qual;3.42.0 from central in [default]
	org.postgresql#postgresql;42.7.3 from central 

# Создание слоев

In [10]:
spark.sql("SHOW CATALOGS").show(truncate=False)

+-------------+
|catalog      |
+-------------+
|hive_cat     |
|spark_catalog|
+-------------+



In [9]:
 spark.sql("SHOW NAMESPACES IN hive_cat").show(truncate=False)

+---------+
|namespace|
+---------+
|cleaned  |
|default  |
|features |
|marts    |
|raw      |
+---------+



In [2]:
 spark.sql("create NAMESPACE if not exists hive_cat.raw")

DataFrame[]

In [5]:
 spark.sql("create NAMESPACE if not exists hive_cat.cleaned")

DataFrame[]

In [6]:
 spark.sql("create NAMESPACE if not exists hive_cat.features")

DataFrame[]

In [ ]:
 spark.sql("create NAMESPACE if not exists hive_cat.marts")

# Загрузка данных

In [5]:
def process_csv_to_iceberg(csv_path, table_name):
    print(f"Reading CSV from: {csv_path}")
    
    df_csv = (spark.read
              .option("header", "true")
              .option("inferSchema", "true")
              .csv(csv_path)
    )
    
    print(f"Writing to Iceberg table: {table_name}")
    
    (df_csv.writeTo(table_name)
     .using("iceberg")
     .createOrReplace()
    )
    
    print(f"First 5 rows from {table_name}:")
    spark.table(table_name).show(5, truncate=False)
    
    return df_csv

In [6]:
process_csv_to_iceberg("/data/test_transaction.csv", "hive_cat.raw.test_transaction")

Reading CSV from: /data/test_transaction.csv


Writing to Iceberg table: hive_cat.raw.test_transaction


25/12/08 13:27:38 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

First 5 rows from hive_cat.raw.test_transaction:


+-------------+-------------+--------------+---------+-----+-----+-----+----------+-----+-----+-----+-----+------+-----+-------------+-------------+---+---+---+---+---+---+---+---+---+---+---+---+-----+---+-----+-----+----+-----+----+----+----+----+----+-----+-----+----+----+----+-----+---+---+---+----+----+---+----+----+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----+----+----+----+-----+----+----+--

DataFrame[TransactionID: int, TransactionDT: int, TransactionAmt: double, ProductCD: string, card1: int, card2: double, card3: double, card4: string, card5: double, card6: string, addr1: double, addr2: double, dist1: double, dist2: double, P_emaildomain: string, R_emaildomain: string, C1: double, C2: double, C3: double, C4: double, C5: double, C6: double, C7: double, C8: double, C9: double, C10: double, C11: double, C12: double, C13: double, C14: double, D1: double, D2: double, D3: double, D4: double, D5: double, D6: double, D7: double, D8: double, D9: double, D10: double, D11: double, D12: double, D13: double, D14: double, D15: double, M1: string, M2: string, M3: string, M4: string, M5: string, M6: string, M7: string, M8: string, M9: string, V1: double, V2: double, V3: double, V4: double, V5: double, V6: double, V7: double, V8: double, V9: double, V10: double, V11: double, V12: double, V13: double, V14: double, V15: double, V16: double, V17: double, V18: double, V19: double, V20: doub

In [7]:
process_csv_to_iceberg("/data/test_identity.csv", "hive_cat.raw.test_identity")

Reading CSV from: /data/test_identity.csv


Writing to Iceberg table: hive_cat.raw.test_identity


First 5 rows from hive_cat.raw.test_identity:
+-------------+-----+--------+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+-----+------+-----+--------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+-------------+-----------------------+-----+--------+--------------+-----+-----+-----+-----+----------+---------------------------+
|TransactionID|id-01|id-02   |id-03|id-04|id-05|id-06|id-07|id-08|id-09|id-10|id-11|id-12   |id-13|id-14 |id-15|id-16   |id-17|id-18|id-19|id-20|id-21|id-22|id-23|id-24|id-25|id-26|id-27|id-28|id-29   |id-30        |id-31                  |id-32|id-33   |id-34         |id-35|id-36|id-37|id-38|DeviceType|DeviceInfo                 |
+-------------+-----+--------+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+-----+------+-----+--------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+-------------+-----------------------+-----+--------+--------------+-----+-----+----

DataFrame[TransactionID: int, id-01: double, id-02: double, id-03: double, id-04: double, id-05: double, id-06: double, id-07: double, id-08: double, id-09: double, id-10: double, id-11: double, id-12: string, id-13: double, id-14: double, id-15: string, id-16: string, id-17: double, id-18: double, id-19: double, id-20: double, id-21: double, id-22: double, id-23: string, id-24: double, id-25: double, id-26: double, id-27: string, id-28: string, id-29: string, id-30: string, id-31: string, id-32: double, id-33: string, id-34: string, id-35: string, id-36: string, id-37: string, id-38: string, DeviceType: string, DeviceInfo: string]

In [8]:
process_csv_to_iceberg("/data/train_transaction.csv", "hive_cat.raw.train_transaction")

Reading CSV from: /data/train_transaction.csv


Writing to Iceberg table: hive_cat.raw.train_transaction


First 5 rows from hive_cat.raw.train_transaction:
+-------------+-------+-------------+--------------+---------+-----+-----+-----+----------+-----+------+-----+-----+-----+-----+-------------+-------------+---+---+---+---+---+---+---+---+---+---+---+---+----+---+-----+-----+----+----+----+----+----+----+----+----+-----+----+----+----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---+----+----+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+

DataFrame[TransactionID: int, isFraud: int, TransactionDT: int, TransactionAmt: double, ProductCD: string, card1: int, card2: double, card3: double, card4: string, card5: double, card6: string, addr1: double, addr2: double, dist1: double, dist2: double, P_emaildomain: string, R_emaildomain: string, C1: double, C2: double, C3: double, C4: double, C5: double, C6: double, C7: double, C8: double, C9: double, C10: double, C11: double, C12: double, C13: double, C14: double, D1: double, D2: double, D3: double, D4: double, D5: double, D6: double, D7: double, D8: double, D9: double, D10: double, D11: double, D12: double, D13: double, D14: double, D15: double, M1: string, M2: string, M3: string, M4: string, M5: string, M6: string, M7: string, M8: string, M9: string, V1: double, V2: double, V3: double, V4: double, V5: double, V6: double, V7: double, V8: double, V9: double, V10: double, V11: double, V12: double, V13: double, V14: double, V15: double, V16: double, V17: double, V18: double, V19: dou

In [9]:
process_csv_to_iceberg("/data/train_identity.csv", "hive_cat.raw.train_identity")

Reading CSV from: /data/train_identity.csv
Writing to Iceberg table: hive_cat.raw.train_identity


First 5 rows from hive_cat.raw.train_identity:
+-------------+-----+--------+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+-----+------+-----+--------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+----------------+-------------------+-----+---------+--------------+-----+-----+-----+-----+----------+-----------------------------+
|TransactionID|id_01|id_02   |id_03|id_04|id_05|id_06|id_07|id_08|id_09|id_10|id_11|id_12   |id_13|id_14 |id_15|id_16   |id_17|id_18|id_19|id_20|id_21|id_22|id_23|id_24|id_25|id_26|id_27|id_28|id_29   |id_30           |id_31              |id_32|id_33    |id_34         |id_35|id_36|id_37|id_38|DeviceType|DeviceInfo                   |
+-------------+-----+--------+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+-----+------+-----+--------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+----------------+-------------------+-----+---------+--------------+-----+-----

DataFrame[TransactionID: int, id_01: double, id_02: double, id_03: double, id_04: double, id_05: double, id_06: double, id_07: double, id_08: double, id_09: double, id_10: double, id_11: double, id_12: string, id_13: double, id_14: double, id_15: string, id_16: string, id_17: double, id_18: double, id_19: double, id_20: double, id_21: double, id_22: double, id_23: string, id_24: double, id_25: double, id_26: double, id_27: string, id_28: string, id_29: string, id_30: string, id_31: string, id_32: double, id_33: string, id_34: string, id_35: string, id_36: string, id_37: string, id_38: string, DeviceType: string, DeviceInfo: string]

# Запись из raw в cleaned

In [4]:
spark.conf.set("spark.sql.shuffle.partitions", "64")
spark.conf.set("spark.sql.files.maxRecordsPerFile", "500000")
#spark.conf.set("parquet.enable.dictionary", "false")

In [11]:
from pyspark.sql import SparkSession, functions as F
import datetime as dt


def load_raw_to_cleaned(spark: SparkSession) -> None:

    train_txn = spark.table("hive_cat.raw.train_transaction")
    train_id = spark.table("hive_cat.raw.train_identity")
    test_txn = spark.table("hive_cat.raw.test_transaction")
    test_id = spark.table("hive_cat.raw.test_identity")

    base_epoch = int(dt.datetime(2017, 1, 1, 0, 0, 0).timestamp())

    def build_view(df_txn, df_id, dataset_label: str, has_target: bool):
        df = (df_txn.alias("t")
            .join(df_id.alias("i"), on="TransactionID", how="left")
            .withColumn(
                "timestamp",
                F.to_timestamp(F.col("TransactionDT").cast("long") + F.lit(base_epoch))
            )
            .withColumnRenamed("TransactionID", "transaction_id")
            .withColumnRenamed("TransactionAmt", "amount")
        )

        if has_target and "isFraud" in df.columns:
            df = df.withColumnRenamed("isFraud", "is_fraud")
        else:
            df = df.withColumn("is_fraud", F.lit(None).cast("int"))

        df = (df
            .withColumn("customer_id", F.col("card1").cast("string"))
            .withColumn(
                "card_id",
                F.concat_ws(
                    "-",
                    F.col("card1").cast("string"),
                    F.col("card2").cast("string"),
                    F.col("card3").cast("string"),
                )
            )
            .withColumn("country", F.col("addr2").cast("string"))
            .withColumn("region", F.col("addr1").cast("string"))
            .withColumn("device_id", F.coalesce(F.col("DeviceInfo"), F.lit("unknown")))
            .withColumn(
                "channel",
                F.when(F.col("DeviceType") == "mobile", "MOBILE")
                 .when(F.col("DeviceType") == "desktop", "WEB")
                 .otherwise("UNKNOWN")
            )
            .withColumn("email", F.col("P_emaildomain"))
            .withColumn("dataset", F.lit(dataset_label))
            .withColumn("event_date", F.to_date(F.col("timestamp")))
        )

        important_cols = [
            "transaction_id",
            "timestamp",
            "event_date",
            "amount",
            "customer_id",
            "card_id",
            "country",
            "region",
            "device_id",
            "channel",
            "email",
            "is_fraud",
            "dataset",
        ]

        cols_to_keep = [c for c in important_cols if c in df.columns]
        df = df.select(*cols_to_keep)

        return df

    train_clean = build_view(train_txn, train_id, dataset_label="train", has_target=True)
    test_clean  = build_view(test_txn,  test_id,  dataset_label="test",  has_target=False)

    spark.sql("CREATE NAMESPACE IF NOT EXISTS hive_cat.cleaned")

    (
        train_clean
        .repartition("event_date")  # repartition(64, "event_date")
        .writeTo("hive_cat.cleaned.transactions")
        .using("iceberg")
        .tableProperty("write.format.default", "parquet")
        .tableProperty("format-version", "2")
        .partitionedBy("event_date")
        .createOrReplace()
    )
    
    (
        test_clean
        .repartition("event_date")
        .writeTo("hive_cat.cleaned.ieee_cis_transactions")
        .append()
    )

In [12]:
if __name__ == "__main__":
    load_raw_to_cleaned(spark)

# Запись из cleaned в слой features

In [5]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.window import Window

In [7]:
def build_features(spark: SparkSession) -> None:
    df = spark.table("hive_cat.cleaned.transactions")

    df = df.filter(F.col("customer_id").isNotNull())

    ts_long = F.col("timestamp").cast("long")

    w_cust_1d = (
        Window.partitionBy("customer_id")
        .orderBy(ts_long)
        .rangeBetween(-86400, 0)
    )

    w_cust_7d = (
        Window.partitionBy("customer_id")
        .orderBy(ts_long)
        .rangeBetween(-86400 * 7, 0)
    )

    w_cust_order = (
        Window.partitionBy("customer_id")
        .orderBy("timestamp")
    )

    df_feat = (
        df
        .withColumn("hour", F.hour("timestamp"))
        .withColumn(
            "is_night",
            F.when((F.col("hour") < 6) | (F.col("hour") >= 23), F.lit(1)).otherwise(F.lit(0))
        )
        .withColumn("log_amount", F.log1p(F.col("amount")))
        .withColumn("cust_txn_cnt_1d", F.count("*").over(w_cust_1d))
        .withColumn("cust_amt_sum_1d", F.sum("amount").over(w_cust_1d))
        .withColumn("cust_txn_cnt_7d", F.count("*").over(w_cust_7d))
        .withColumn("cust_amt_sum_7d", F.sum("amount").over(w_cust_7d))
        .withColumn("prev_timestamp", F.lag("timestamp").over(w_cust_order))
        .withColumn(
            "secs_since_prev_txn",
            (F.col("timestamp").cast("long") - F.col("prev_timestamp").cast("long"))
        )
    )

    cols = [
        "transaction_id",
        "timestamp",
        "event_date",
        "dataset",
        "customer_id",
        "card_id",
        "country",
        "region",
        "device_id",
        "channel",
        "email",
        "is_fraud",
        "amount",
        "log_amount",
        "hour",
        "is_night",
        "cust_txn_cnt_1d",
        "cust_amt_sum_1d",
        "cust_txn_cnt_7d",
        "cust_amt_sum_7d",
        "secs_since_prev_txn",
    ]

    df_feat = df_feat.select(*cols)

    (
        df_feat
        .repartition("event_date")
        .writeTo("hive_cat.features.transaction_features")
        .using("iceberg")
        .tableProperty("write.format.default", "parquet")
        .tableProperty("format-version", "2")
        .partitionedBy("event_date")
        .createOrReplace()
    )

25/12/09 07:12:06 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
                                                                                

In [ ]:
if __name__ == "__main__":
    build_ieee_features(spark)

# Данные для модели

## LightGBM

### export train

In [3]:
from pyspark.sql import SparkSession, functions as F

In [10]:
def export_train_dataset(spark: SparkSession) -> None:
    df = spark.table("hive_cat.features.transaction_features") # fix

    df_train = df.filter(
        (F.col("dataset") == "train") & F.col("is_fraud").isNotNull()
    )

    output_path = "ml_dataset/dataset_train" # добавить нормальный путь

    (
        df_train
        .repartition(1)
        .write
        .mode("overwrite")
        .parquet(output_path)
    )

    print(f"Train dataset exported to {output_path}")

In [11]:
if __name__ == "__main__":
    export_train_dataset(spark)

Train dataset exported to hive_cat.features.ml_dataset


### export test

In [4]:
def export_test_dataset(spark: SparkSession) -> None:
    df = spark.table("hive_cat.features.ieee_cis_transaction_features") # fix

    df_train = df.filter(
        (F.col("dataset") == "test")
    )

    output_path = "ml_dataset/dataset_test" # добавить нормальный путь

    (
        df_train
        .repartition(1)
        .write
        .mode("overwrite")
        .parquet(output_path)
    )

    print(f"Train dataset exported to {output_path}")

In [5]:
if __name__ == "__main__":
    export_test_dataset(spark)

Train dataset exported to ml_dataset/dataset_test


## GNN

### Подготовка ребер и узлов

In [2]:
from pyspark.sql import SparkSession, functions as F

In [3]:
def export_gnn_graph(spark: SparkSession,base_path: str = "ml_dataset/gnn_edge") -> None:
    df = spark.table("hive_cat.features.ieee_cis_transaction_features")

    df_train = df.filter(F.col("dataset") == F.lit("train"))

    df_train = (
        df_train
        .withColumn("transaction_id_str", F.col("transaction_id").cast("string"))
        .withColumn("customer_id_str", F.col("customer_id").cast("string"))
        .withColumn("card_id_str", F.col("card_id").cast("string"))
        .withColumn("device_id_str", F.col("device_id").cast("string"))
        .withColumn("email_str", F.col("email").cast("string"))
    )

    df_train = df_train.withColumn(
        "label",
        F.col("is_fraud").cast("int")
    )

    # -------- NODES --------

    customers = (
        df_train
        .select(F.col("customer_id_str").alias("customer_id"))
        .where(F.col("customer_id_str").isNotNull())
        .distinct()
    )
    customers.coalesce(1).write.mode("overwrite").parquet(f"{base_path}/nodes/customers")

    cards = (
        df_train
        .select(F.col("card_id_str").alias("card_id"))
        .where(F.col("card_id_str").isNotNull())
        .distinct()
    )
    cards.coalesce(1).write.mode("overwrite").parquet(f"{base_path}/nodes/cards")

    devices = (
        df_train
        .select(F.col("device_id_str").alias("device_id"))
        .where(F.col("device_id_str").isNotNull())
        .distinct()
    )
    devices.coalesce(1).write.mode("overwrite").parquet(f"{base_path}/nodes/devices")

    emails = (
        df_train
        .select(F.col("email_str").alias("email"))
        .where(F.col("email_str").isNotNull())
        .distinct()
    )
    emails.coalesce(1).write.mode("overwrite").parquet(f"{base_path}/nodes/emails")

    txns = (
        df_train
        .select(F.col("transaction_id_str").alias("transaction_id"))
        .where(F.col("transaction_id_str").isNotNull())
        .distinct()
    )
    txns.coalesce(1).write.mode("overwrite").parquet(f"{base_path}/nodes/transactions")

    # -------- LABELS --------

    txn_labels = (
        df_train
        .select(
            F.col("transaction_id_str").alias("transaction_id"),
            F.col("label")
        )
        .where(F.col("transaction_id_str").isNotNull())
    )
    txn_labels.coalesce(1).write.mode("overwrite").parquet(f"{base_path}/labels/transactions") # coalesce -> repartition

    # -------- EDGES --------

    edges_cust_card = (
        df_train
        .select(
            F.col("customer_id_str").alias("customer_id"),
            F.col("card_id_str").alias("card_id")
        )
        .where(
            F.col("customer_id_str").isNotNull() &
            F.col("card_id_str").isNotNull()
        )
        .distinct()
    )
    edges_cust_card.coalesce(1).write.mode("overwrite").parquet(f"{base_path}/edges/customer_card")

    edges_card_device = (
        df_train
.select(
            F.col("card_id_str").alias("card_id"),
            F.col("device_id_str").alias("device_id")
        )
        .where(
            F.col("card_id_str").isNotNull() &
            F.col("device_id_str").isNotNull()
        )
        .distinct()
    )
    edges_card_device.coalesce(1).write.mode("overwrite").parquet(f"{base_path}/edges/card_device")

    edges_card_email = (
        df_train
        .select(
            F.col("card_id_str").alias("card_id"),
            F.col("email_str").alias("email")
        )
        .where(
            F.col("card_id_str").isNotNull() &
            F.col("email_str").isNotNull()
        )
        .distinct()
    )
    edges_card_email.coalesce(1).write.mode("overwrite").parquet(f"{base_path}/edges/card_email")

    edges_cust_txn = (
        df_train
        .select(
            F.col("customer_id_str").alias("customer_id"),
            F.col("transaction_id_str").alias("transaction_id")
        )
        .where(
            F.col("customer_id_str").isNotNull() &
            F.col("transaction_id_str").isNotNull()
        )
        .distinct()
    )
    edges_cust_txn.coalesce(1).write.mode("overwrite").parquet(f"{base_path}/edges/customer_txn")

    edges_txn_device = (
        df_train
        .select(
            F.col("transaction_id_str").alias("transaction_id"),
            F.col("device_id_str").alias("device_id")
        )
        .where(
            F.col("transaction_id_str").isNotNull() &
            F.col("device_id_str").isNotNull()
        )
        .distinct()
    )
    edges_txn_device.coalesce(1).write.mode("overwrite").parquet(f"{base_path}/edges/txn_device")

    print("GNN graph export finished")

In [4]:
if __name__ == "__main__":
    export_gnn_graph(spark)

GNN graph export finished


# Запись в marts

In [11]:
def import_scored_to_marts(spark: SparkSession) -> None:
    scored_path = "../models/test_scored"

    df_scored = spark.read.parquet(scored_path)

    (
        df_scored
        .repartition("event_date")
        .writeTo("hive_cat.marts.scored_transactions")
        .using("iceberg")
        .tableProperty("write.format.default", "parquet")
        .tableProperty("format-version", "2")
        .partitionedBy("event_date")
        .createOrReplace()
    )

    print("Scored data imported into hive_cat.marts.scored_transactions")

In [12]:
if __name__ == "__main__":
    import_scored_to_marts(spark)

Scored data imported into hive_cat.marts.scored_transactions


# Кластеризация

In [2]:
from pyspark.sql import SparkSession, functions as F
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.clustering import KMeans, BisectingKMeans

In [3]:
df = spark.table("hive_cat.marts.scored_transactions") \
    .select(
        "transaction_id",
        "log_amount",
        F.log10(F.col("secs_since_prev_txn") + 1).alias("log_secs_prev"),
        F.log10(F.col("cust_txn_cnt_7d") + 1).alias("log_txn_cnt_7d"),
        "hour",
        "is_night",
        "fraud_score",
        "fraud_label"
    ) \
    .na.fill(0)

In [4]:
assembler = VectorAssembler(
    inputCols=["log_amount", "log_secs_prev", "log_txn_cnt_7d", "hour", "is_night"],
    outputCol="features_raw"
)

df2 = assembler.transform(df)

In [5]:
scaler = StandardScaler(inputCol="features_raw", outputCol="features")
model_scaler = scaler.fit(df2)
df3 = model_scaler.transform(df2)

In [6]:
k = 6
kmeans = BisectingKMeans(k=k, featuresCol="features", predictionCol="cluster")
model_kmeans = kmeans.fit(df3)

df_clusters = model_kmeans.transform(df3)

25/12/10 12:18:21 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
                                                                                

In [7]:
(
    df_clusters
    .select(
        "transaction_id",
        "cluster",
        "log_amount",
        "log_secs_prev",
        "log_txn_cnt_7d",
        "hour",
        "is_night",
        "fraud_score",
        "fraud_label"
    )
    .writeTo("hive_cat.marts.scored_transactions_clustered")
    .using("iceberg")
    .createOrReplace()
)